# Building LangChain Agent Base - Tutorial

This notebook demonstrates **how to build** the LangChain Agent Base tool using LangChain 1.0 patterns.

## 🎯 **What You'll Learn:**
- **LangChain 1.0 Patterns**: `create_agent()`, tools, RAG, HITL workflows
- **Architecture Design**: How we built the class-based system
- **Tool Creation**: Building reusable tools with `@tool` decorator
- **RAG Implementation**: Document indexing and retrieval
- **Multi-Agent Systems**: Supervisor and delegation patterns

## 📚 **This is the "How It's Built" Tutorial**
For **"How to Use It"**, see the **"Agent Usage Examples.ipynb"** notebook.

## 🏗️ Architecture Overview

This tutorial shows how we built the LangChain Agent Base tool using **LangChain 1.0 best practices**.

### 🔧 **Core Components We'll Build:**
- **`src/base.py`**: Agent classes wrapping `create_agent()`
- **`src/tools.py`**: Reusable tools with `@tool` decorator  
- **`src/rag.py`**: RAG system with HuggingFace embeddings
- **`src/commands.py`**: Direct command execution system
- **`src/agent.py`**: Original tutorial compatibility layer

### 🎓 **LangChain 1.0 Concepts:**
- **`create_agent()`**: Modern agent creation workflow
- **Tool Integration**: Adding tools to agents seamlessly  
- **Human-in-the-Loop**: Approval workflows with middleware
- **Multi-Agent Patterns**: Agents as tools for delegation
- **RAG Tools**: Retrieval-augmented generation as agent tools

Let's build it step by step!

# Step 1: Building Basic Tools

The foundation of any agent system is **tools**. Let's start by creating tools using LangChain's `@tool` decorator.

## 🛠️ **Tool Design Principles:**
1. **Single Responsibility**: Each tool does one thing well
2. **Clear Signatures**: Type hints for parameters and returns
3. **Good Docstrings**: Describe what the tool does for the LLM
4. **Error Handling**: Graceful failure with helpful messages

Let's build some example tools from scratch:

## 1. Setup and Installation
Let's get our environment ready by `uv sync`ing!

## 2. Environment Variables
We need to set our OpenAI API Key. We also enable LangSmith tracing here.

In [ ]:
# Building Tools with LangChain @tool decorator
from langchain_core.tools import tool
import math
import random

# Tool 1: Basic calculator
@tool
def basic_calculator(expression: str) -> str:
    """Evaluate basic mathematical expressions safely."""
    try:
        # Only allow safe mathematical operations
        allowed_chars = set('0123456789+-*/().= ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Only basic math operations allowed"
        
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Tool 2: Weather simulator (mock API)
@tool
def get_weather_info(location: str) -> str:
    """Get current weather information for a location."""
    weather_conditions = ["Sunny", "Cloudy", "Rainy", "Snowy", "Foggy"]
    temperature = random.randint(15, 30)
    condition = random.choice(weather_conditions)
    
    return f"Weather in {location}: {condition}, {temperature}°C"

# Tool 3: Text analyzer
@tool
def analyze_text(text: str) -> str:
    """Analyze text and return statistics."""
    words = len(text.split())
    chars = len(text)
    sentences = text.count('.') + text.count('!') + text.count('?')
    
    return f"""Text Analysis:
- Words: {words}
- Characters: {chars}  
- Sentences: {sentences}
- Average words per sentence: {words/max(1, sentences):.1f}"""

print("✅ Created 3 tools with @tool decorator")
print("📋 Available tools:")
for tool_func in [basic_calculator, get_weather_info, analyze_text]:
    print(f"  - {tool_func.name}: {tool_func.description}")

# Test the tools
print("\n🧪 Testing tools directly:")
print("Calculator:", basic_calculator("2 + 3 * 4"))
print("Weather:", get_weather_info("Paris"))
print("Text Analysis:", analyze_text("Hello world! This is a test."))

In [ ]:
# Path setup for tutorial - Add parent directory to access src/ module
import sys
from pathlib import Path

# Add parent directory to path so we can import from src/ when needed
parent_dir = Path('.').parent.resolve()
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

print(f"✅ Tutorial setup: Added {parent_dir} to Python path")

# Step 2: Creating Agents with create_agent()

Now let's build agents using LangChain 1.0's `create_agent()` function. This is the modern way to create agents.

## 🤖 **Agent Creation Pattern:**
1. **Choose a model** (we'll use Groq for speed)
2. **Select tools** to give the agent capabilities
3. **Define system prompt** to set behavior
4. **Use `create_agent()`** to create the agent graph

In [ ]:
# Step 2: Building agents with create_agent()
import os
import getpass

# Setup environment
def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

# Using Groq for fast inference
_set_if_undefined("GROQ_API_KEY")

# Import LangChain components
from langchain_groq import ChatGroq
from langchain.agents import create_agent

# Step 1: Create the model
def get_model():
    """Get the LLM model for our agents."""
    return ChatGroq(model="openai/gpt-oss-120b", temperature=0)

# Step 2: Build a simple agent
def build_simple_agent():
    """Build an agent using LangChain 1.0 create_agent() pattern."""
    model = get_model()
    
    # Select tools for this agent
    tools = [basic_calculator, get_weather_info]
    
    # Create the agent with create_agent()
    agent = create_agent(
        model=model,
        tools=tools,
        system_prompt="You are a helpful assistant with calculator and weather tools."
    )
    
    return agent

# Create and test the agent
print("🤖 Building agent with create_agent()...")
try:
    simple_agent = build_simple_agent()
    print("✅ Agent created successfully!")
    
    # Test the agent
    response = simple_agent.invoke({
        "messages": [{"role": "user", "content": "What's 15 * 8? Also what's the weather in Tokyo?"}]
    })
    print(f"Agent Response: {response['messages'][-1].content}")
    
except Exception as e:
    print(f"Expected error (need API key): {e}")
    print("💡 This shows how create_agent() works - model + tools + prompt = agent")

# Step 3: Building the Agent Class Wrapper

Now let's build a **class-based wrapper** around `create_agent()` to make it easier to use and extend.

## 🏗️ **Class Design Goals:**
1. **Wrap create_agent()**: Hide complexity, expose simplicity
2. **Dynamic tool management**: Add/remove tools easily  
3. **Conversation interface**: Simple `.chat()` method
4. **Extensibility**: Easy to subclass and customize

In [ ]:
# Step 3: Building the Agent class wrapper
from typing import List, Callable
from langchain_core.messages import HumanMessage

class SimpleAgent:
    """
    A simple wrapper around create_agent() for easier use.
    This is the core pattern used in src/base.py
    """
    
    def __init__(self, 
                 model_name: str = "openai/gpt-oss-120b",
                 system_prompt: str = "You are a helpful AI assistant.",
                 initial_tools: List[Callable] = None):
        """Initialize the agent."""
        self.model_name = model_name
        self.system_prompt = system_prompt
        self.tools = initial_tools or []
        self.model = ChatGroq(model=model_name, temperature=0)
        self.agent = None
        self._rebuild_agent()
    
    def _rebuild_agent(self):
        """Rebuild the agent when tools change."""
        if self.tools:
            self.agent = create_agent(
                model=self.model,
                tools=self.tools,
                system_prompt=self.system_prompt
            )
        else:
            # No tools = basic chat model
            self.agent = self.model
    
    def add_tool(self, tool_func: Callable):
        """Add a tool to this agent."""
        self.tools.append(tool_func)
        self._rebuild_agent()
        print(f"✅ Added tool: {tool_func.name}")
    
    def add_tools(self, tools: List[Callable]):
        """Add multiple tools at once."""
        self.tools.extend(tools)
        self._rebuild_agent()
        print(f"✅ Added {len(tools)} tools")
    
    def list_tools(self) -> List[str]:
        """List current tool names."""
        return [tool.name for tool in self.tools]
    
    def chat(self, message: str) -> str:
        """Chat with the agent."""
        try:
            if self.agent and hasattr(self.agent, 'invoke'):
                # Agent with tools
                response = self.agent.invoke({
                    "messages": [HumanMessage(content=message)]
                })
                return response['messages'][-1].content
            else:
                # Basic model without tools
                response = self.model.invoke([HumanMessage(content=message)])
                return response.content
        except Exception as e:
            return f"Error: {str(e)}"

# Test the Agent class
print("🏗️ Building Agent class wrapper...")

# Create agent with initial tools
agent = SimpleAgent(
    system_prompt="You are a helpful assistant with math and weather capabilities.",
    initial_tools=[basic_calculator, get_weather_info]
)

print(f"Agent created with tools: {agent.list_tools()}")

# Add another tool
agent.add_tool(analyze_text)

print(f"Updated tools: {agent.list_tools()}")

# Test the chat interface
print("\n💬 Testing chat interface:")
try:
    response = agent.chat("Calculate 25 * 4 and tell me about the weather in London")
    print(f"Response: {response}")
except Exception as e:
    print(f"Expected error: {e}")
    print("💡 This shows the Agent class pattern that wraps create_agent()")

# Step 4: Building RAG (Retrieval-Augmented Generation)

Let's build a **RAG system** that can search documents and provide that capability as tools to agents.

## 📚 **RAG Architecture:**
1. **Document Loading**: Load text from various sources  
2. **Text Splitting**: Break documents into chunks
3. **Embeddings**: Convert text to vector representations
4. **Vector Storage**: Store embeddings in a database
5. **Retrieval Tools**: Create tools that search the documents

In [ ]:
# Step 4: Building RAG system
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.tools import create_retriever_tool

# Mock embeddings for demo (in real code, use HuggingFaceEmbeddings)
class MockEmbeddings:
    """Mock embeddings for demonstration."""
    def embed_documents(self, texts):
        return [[0.1, 0.2, 0.3] for _ in texts]
    
    def embed_query(self, text):
        return [0.1, 0.2, 0.3]

# Mock vector store for demo
class MockVectorStore:
    """Mock vector store for demonstration."""
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings
        
    def similarity_search(self, query, k=3):
        # In real implementation, this would do semantic search
        # For demo, return first few documents
        return self.documents[:k]
    
    def as_retriever(self):
        return MockRetriever(self)

class MockRetriever:
    """Mock retriever for demonstration."""
    def __init__(self, vector_store):
        self.vector_store = vector_store
    
    def invoke(self, query):
        return self.vector_store.similarity_search(query)

def build_rag_system(documents_text: List[str]):
    """
    Build a RAG system from documents.
    This is the pattern used in src/rag.py
    """
    print("🔍 Building RAG system...")
    
    # Step 1: Create document objects
    documents = [Document(page_content=text) for text in documents_text]
    print(f"✅ Created {len(documents)} documents")
    
    # Step 2: Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    
    split_docs = text_splitter.split_documents(documents)
    print(f"✅ Split into {len(split_docs)} chunks")
    
    # Step 3: Create embeddings and vector store
    embeddings = MockEmbeddings()
    vector_store = MockVectorStore(split_docs, embeddings)
    print("✅ Created embeddings and vector store")
    
    # Step 4: Create retriever tool
    retriever = vector_store.as_retriever()
    
    search_tool = create_retriever_tool(
        retriever=retriever,
        name="search_documents",
        description="Search through the document collection for relevant information."
    )
    
    print("✅ Created retriever tool")
    return search_tool

# Test RAG system
sample_documents = [
    "Python is a high-level programming language known for its simplicity and readability. It's widely used in data science, web development, and automation.",
    "Machine learning is a subset of artificial intelligence that enables computers to learn and make decisions from data without explicit programming.",
    "LangChain is a framework for developing applications powered by language models. It provides tools for chaining together different components.",
    "Vector databases store and retrieve data based on semantic similarity rather than exact matches, making them ideal for AI applications."
]

# Build RAG system
rag_tool = build_rag_system(sample_documents)

print(f"\n📋 RAG tool created: {rag_tool.name}")
print(f"Description: {rag_tool.description}")

# Test RAG tool
print("\n🔎 Testing RAG search:")
try:
    results = rag_tool.invoke("What is Python used for?")
    print("Search results:", [doc.page_content for doc in results])
except Exception as e:
    print(f"Demo search: {e}")
    print("💡 This shows how RAG tools are created and can be added to agents")

# Step 5: Multi-Agent Systems

Let's build a **multi-agent system** where agents can work together by treating other agents as tools.

## 🤝 **Multi-Agent Pattern:**
1. **Create specialized agents** with different capabilities
2. **Wrap agents as tools** using `@tool` decorator
3. **Create supervisor agent** that delegates to specialists
4. **Handle complex queries** requiring multiple agents

In [ ]:
# Step 5: Building Multi-Agent Systems

def build_multi_agent_system():
    """
    Build a multi-agent system with specialized agents.
    This demonstrates the pattern used in src/base.py
    """
    print("🤝 Building multi-agent system...")
    
    # Create specialized agents
    math_agent = SimpleAgent(
        system_prompt="You are a math specialist. Focus on calculations and mathematical problem solving.",
        initial_tools=[basic_calculator]
    )
    
    research_agent = SimpleAgent(
        system_prompt="You are a research specialist. Focus on finding and analyzing information.",
        initial_tools=[analyze_text, rag_tool]  # Add RAG tool for document search
    )
    
    # Wrap agents as tools for the supervisor
    @tool
    def delegate_to_math(query: str) -> str:
        """Delegate mathematical questions to the math specialist."""
        try:
            return f"Math Agent: {math_agent.chat(query)}"
        except Exception as e:
            return f"Math Agent Error: {str(e)}"
    
    @tool  
    def delegate_to_research(query: str) -> str:
        """Delegate research questions to the research specialist."""
        try:
            return f"Research Agent: {research_agent.chat(query)}"
        except Exception as e:
            return f"Research Agent Error: {str(e)}"
    
    # Create supervisor agent with delegation tools
    supervisor = SimpleAgent(
        system_prompt="""You are a supervisor agent. You coordinate between specialists:
        - Use delegate_to_math for calculations and math problems
        - Use delegate_to_research for information lookup and analysis
        - You can use both agents for complex queries requiring multiple steps""",
        initial_tools=[delegate_to_math, delegate_to_research]
    )
    
    print("✅ Created math agent, research agent, and supervisor")
    return supervisor, math_agent, research_agent

# Build the multi-agent system
supervisor, math_agent, research_agent = build_multi_agent_system()

print(f"\n📋 Supervisor tools: {supervisor.list_tools()}")
print(f"Math agent tools: {math_agent.list_tools()}")  
print(f"Research agent tools: {research_agent.list_tools()}")

# Test multi-agent coordination
print("\n🎯 Testing multi-agent coordination:")
complex_query = "Calculate 25 * 16, then analyze the text 'Machine learning is transforming industries' and tell me what documents mention Python"

try:
    response = supervisor.chat(complex_query)
    print(f"Supervisor Response: {response}")
except Exception as e:
    print(f"Expected error: {e}")
    print("💡 This shows how agents can be composed as tools for delegation")

print("\n🏗️ Multi-Agent Architecture:")
print("1. Specialist agents have domain-specific tools")
print("2. Agents are wrapped as @tool functions") 
print("3. Supervisor agent uses delegation tools")
print("4. Complex queries are routed to appropriate specialists")

# Step 6: Command System

Finally, let's build a **command system** that allows direct execution of tools without going through the chat interface.

## ⚡ **Command System Benefits:**
1. **Direct Execution**: Skip conversation for quick tasks
2. **Batch Processing**: Run multiple commands efficiently  
3. **Scripting**: Integrate agents into automated workflows
4. **Performance**: Faster than chat for simple operations

In [ ]:
# Step 6: Building Command System
from typing import Dict, Any
from dataclasses import dataclass

@dataclass
class CommandInfo:
    """Information about a command."""
    name: str
    func: Callable
    description: str
    usage: str

def command(name: str, description: str = None, usage: str = None):
    """
    Decorator to register a function as a command.
    This is the pattern used in src/commands.py
    """
    def decorator(func: Callable) -> Callable:
        func._command_info = CommandInfo(
            name=name,
            func=func, 
            description=description or func.__doc__ or "No description",
            usage=usage or f"/{name}"
        )
        return func
    return decorator

class CommandRegistry:
    """Registry for managing and executing commands."""
    
    def __init__(self):
        self.commands: Dict[str, CommandInfo] = {}
        
    def add_command(self, func: Callable):
        """Add a command function to the registry."""
        if hasattr(func, '_command_info'):
            info = func._command_info
            self.commands[info.name] = info
            print(f"✅ Registered command: /{info.name}")
        else:
            print(f"❌ Function {func.__name__} is not decorated with @command")
    
    def execute_command(self, command_str: str, **kwargs) -> str:
        """Execute a command with given arguments."""
        # Parse command (remove leading /)
        cmd_name = command_str.lstrip('/')
        
        if cmd_name not in self.commands:
            return f"Unknown command: {cmd_name}. Available: {list(self.commands.keys())}"
        
        try:
            command_info = self.commands[cmd_name]
            result = command_info.func(**kwargs)
            return str(result)
        except Exception as e:
            return f"Error executing /{cmd_name}: {str(e)}"
    
    def list_commands(self) -> List[str]:
        """List available command names."""
        return list(self.commands.keys())

# Create sample commands
@command("calc", "Quick calculation", "/calc expression=<math_expression>")
def quick_calc(expression: str) -> str:
    """Execute a quick calculation."""
    return basic_calculator(expression)

@command("weather", "Get weather info", "/weather location=<city>") 
def quick_weather(location: str) -> str:
    """Get weather for a location."""
    return get_weather_info(location)

@command("analyze", "Analyze text", "/analyze text=<text_to_analyze>")
def quick_analyze(text: str) -> str:
    """Analyze text statistics."""
    return analyze_text(text)

# Build command system
print("⚡ Building command system...")
registry = CommandRegistry()

# Register commands
for cmd_func in [quick_calc, quick_weather, quick_analyze]:
    registry.add_command(cmd_func)

print(f"\n📋 Available commands: {registry.list_commands()}")

# Test command execution
print("\n🎯 Testing command execution:")
print("Calc:", registry.execute_command("/calc", expression="15 * 8"))
print("Weather:", registry.execute_command("/weather", location="Tokyo"))
print("Analyze:", registry.execute_command("/analyze", text="Hello world! How are you?"))

# Add command system to agents
class AgentWithCommands(SimpleAgent):
    """Agent with command system integration."""
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.commands = CommandRegistry()
        
    def add_command(self, func: Callable):
        """Add a command to this agent."""
        self.commands.add_command(func)
        
    def execute_command(self, command_str: str, **kwargs) -> str:
        """Execute a command directly."""
        return self.commands.execute_command(command_str, **kwargs)

# Test agent with commands
print("\n🤖 Testing agent with commands:")
enhanced_agent = AgentWithCommands()
enhanced_agent.add_command(quick_calc)
enhanced_agent.add_command(quick_weather)

print("Direct command:", enhanced_agent.execute_command("/calc", expression="100 / 4"))
print("💡 Commands provide direct tool execution without chat overhead")

# 🎉 Summary: What We Built

We've just built the **LangChain Agent Base** system from scratch using LangChain 1.0 patterns!

## 🏗️ **Architecture Components:**

### **1. Tools (`@tool` decorator)**
- Basic calculator, weather, text analysis
- Clear signatures and documentation
- Error handling and validation

### **2. Agent Class (wraps `create_agent()`)**
- Dynamic tool management
- Simple chat interface  
- Rebuilds agent when tools change

### **3. RAG System**
- Document loading and chunking
- Vector embeddings and storage
- Retrieval tools for agents

### **4. Multi-Agent System** 
- Specialized agents for different domains
- Agents wrapped as tools for delegation
- Supervisor pattern for coordination

### **5. Command System**
- Direct tool execution without chat
- Command registry and decorators
- Integration with agent classes

## 📂 **This Becomes the `src/` Files:**
- **`tools.py`**: Tool definitions with `@tool`
- **`base.py`**: Agent classes and factories  
- **`rag.py`**: RAG system implementation
- **`commands.py`**: Command system
- **`agent.py`**: Original tutorial compatibility

## 🚀 **Next Steps:**
Check out **"Agent Usage Examples.ipynb"** to see how to **use** these components to build applications!

In [ ]:
# Multi-agent supervisor - agents become tools for delegation
async def demo_multi_agent():
    """Demo the multi-agent supervisor pattern."""
    from src.base import create_multi_agent_supervisor
    
    print("🤝 Creating multi-agent supervisor...")
    
    try:
        # This creates a supervisor with math/science/coding agents as tools
        supervisor = await create_multi_agent_supervisor()
        
        print(f"✅ Supervisor created with {len(supervisor.tools)} delegation tools:")
        for tool_name in supervisor.list_tools():
            print(f"   - {tool_name}")
        
        # Test complex query requiring multiple agents
        print("\n🎯 Testing multi-agent coordination:")
        query = "Calculate the kinetic energy formula KE = 0.5 * m * v² for m=10kg, v=5m/s, then show me Python code to implement this calculation."
        
        response = supervisor.chat(query)
        print(f"Supervisor Response: {response}")
        
    except Exception as e:
        print(f"Expected error without API: {e}")
        print("\n🔧 How multi-agent works:")
        print("1. Create specialized agents (math, science, coding)")
        print("2. Wrap each agent as a @tool function")  
        print("3. Give tools to supervisor agent")
        print("4. Supervisor routes queries to appropriate agents")
        print("5. Can delegate to multiple agents for complex tasks")

await demo_multi_agent()

## 3. The `create_agent` Workflow

LangChain 1.0 introduces `create_agent` as the standard way to build agents. It simplifies the process while retaining the power of LangGraph under the hood.

We've defined a simple agent in `src/agent.py` that has access to a weather tool and a "magic calculator".


In [ ]:
from agent import build_simple_agent

# Build the agent
agent = build_simple_agent()

# Run the agent
response = agent.invoke({"messages": [{"role": "user", "content": "What is the weather in San Francisco?"}]})
print(response["messages"][-1].content)

## 4. Human-in-the-Loop (HITL)

Sometimes we want to approve sensitive actions before they happen. LangChain 1.0 makes this easy with middleware.

We've configured our `hitl_agent` to interrupt before using the `magic_calculator` tool.


In [ ]:
from guide.agent import build_hitl_agent
from langgraph.types import Command
# Import uuid7 for generating valid thread IDs compliant with LangSmith
from langsmith import uuid7

# Build the HITL agent
hitl_agent = build_hitl_agent()

# Configuration for the thread (required for checkpointing)
# Using uuid7 to avoid warnings and ensure best practice
thread_id = str(uuid7())
config = {"configurable": {"thread_id": thread_id}}

print(f"--- Asking to calculate (Thread: {thread_id}) ---")
# We use .stream() to see steps
events = list(hitl_agent.stream(
    {"messages": [{"role": "user", "content": "Please use the magic calculator to add 5 and 5."}]},
    config=config
))

# Print events to verify the initial run and interruption
for i, event in enumerate(events):
    # print(f"Event {i} keys: {event.keys()}")
    if "messages" in event:
        event["messages"][-1].pretty_print()

You should see the tool call request, and then the stream ends (due to interruption). We can now resume.

In [ ]:
# Resume execution
# We need to construct the resume payload dynamically based on the interrupt ID.

# 1. Fetch the current state
state = hitl_agent.get_state(config)

# 2. Find the interrupt
tasks = state.tasks
resume_payload = {}

if tasks and tasks[0].interrupts:
    interrupt = tasks[0].interrupts[0]
    print(f"Found interrupt: {interrupt.id}")
    
    # 3. Construct payload mapping interrupt ID to decision
    resume_payload = {
        interrupt.id: {
            "decisions": [{"type": "approve"}]
        }
    }
else:
    print("No active interrupts found. Agent might have finished.")

# 4. Resume if we have a payload
if resume_payload:
    print(f"--- Resuming (Thread: {thread_id}) ---")
    resume_output = list(hitl_agent.stream(
        Command(resume=resume_payload),
        config=config
    ))
    
    if not resume_output:
        print("No events received after resume. Checking final state...")
        final_state = hitl_agent.get_state(config)
        if final_state.values and "messages" in final_state.values:
             final_state.values["messages"][-1].pretty_print()
    else:
        for event in resume_output:
            if "messages" in event:
                event["messages"][-1].pretty_print()
            else:
                print(event)

## 5. Retrieval (RAG)

Agents become truly powerful when they can access external knowledge. We can add a retriever as a tool.

We've set up a RAG pipeline over the "Musk v Altman" complaint in `src/rag.py`, using OpenAI embeddings and Qdrant.


In [ ]:
from agent import build_rag_agent

rag_agent = build_rag_agent()

response = rag_agent.invoke({
    "messages": [{"role": "user", "content": "When should I use LangChain 1.0?"}]
})

print(response["messages"][-1].content)

## 6. Multi-Agent Systems

LangChain 1.0 agents are graphs, which means they can be composed! We can treat an agent as a tool for another agent.

We have a "Supervisor" that delegates to a "Researcher" (who has RAG access) and a "Writer" (who has a specific persona).


In [ ]:
from agent import build_multi_agent_system

supervisor = build_multi_agent_system()

# This complex query requires research and then writing
query = "When should I use LangChain vs. LangGraph, write a Shakespearean sonnet about that."

response = supervisor.invoke({
    "messages": [{"role": "user", "content": query}]
})

print(response["messages"][-1].content)